Tutorial in Basics of pandas #2 from [User Guide](https://pandas.pydata.org/docs/user_guide/index.html)
@aniafijarczyk

In [ ]:
import pandas as pd
import numpy as np

# Additional useful methods
- correlations
- ranking
- rolling window
- apply
- groupby
- groupby aggregate
- groupby tranform
- groupby filter
- groupby apply
- other useful features
- Excercises

## Correlation

[Correlation](https://pandas.pydata.org/docs/user_guide/computation.html#correlation) may be computed using the corr() method

In [ ]:
frame = pd.DataFrame(np.random.randn(1000, 5), columns=["a", "b", "c", "d", "e"])
frame.iloc[::2] = np.nan
frame
frame["a"].corr(frame["b"])
frame["a"].corr(frame["b"], method="spearman")
# Pairwise correlation of DataFrame columns
frame.corr()

## Data ranking

The rank() method produces a [data ranking](https://pandas.pydata.org/docs/user_guide/computation.html#data-ranking) with ties being assigned the mean of the ranks (by default) for the group

In [ ]:
s = pd.Series(np.random.randn(5), index=list("abcde"))
s["d"] = s["b"]  # so there's a tie
s
s.rank()

In [ ]:
df = pd.DataFrame(np.random.randn(10, 6))
df[4] = df[2][:5]  # some ties
df
df.rank(0) # rows
df.rank(1) # columns

## Apply

Using lambda function

In [ ]:
df[0].apply(lambda x: x * (-1))
df[0].apply(lambda x: x if x>0 else -x)
df[0].apply(lambda x: 1 if x>0 else 0)

Creating new column

In [ ]:
df[6] = df[0].apply(lambda x: 1 if x>0 else 0)
df

Working with multiple columns

In [ ]:
df.apply(lambda x: x[0]*x[1]*x[2]*x[3], axis=1)
df.apply(lambda x: "A" if x[0]>x[1] else "B", axis=1)

## Rolling window

Generic [rolling windows](https://pandas.pydata.org/docs/user_guide/window.html#rolling-window) support specifying windows as a fixed number of observations or variable number of observations based on an offset. If a time based offset is provided, the corresponding time based index must be monotonic. Rolling window [functions](https://pandas.pydata.org/docs/reference/window.html#api-functions-rolling)

In [ ]:
times = ['2020-01-01', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-29']
s = pd.Series(range(5), index=pd.DatetimeIndex(times))
s
# Window with 2 observations
s.rolling(window=2).sum()

# Window with 2 days worth of observations
s.rolling(window='2D').sum()

By default the labels are set to the right edge of the window, but a center keyword is available so the labels can be set at the center.

In [ ]:
s = pd.Series(range(10))
s.rolling(window=5).mean()
s.rolling(window=5, center=True).mean()

The inclusion of the interval endpoints in rolling window calculations can be specified with the closed parameter

In [ ]:
df = pd.DataFrame(
    {"x": 1},
    index=[
        pd.Timestamp("20130101 09:00:01"),
        pd.Timestamp("20130101 09:00:02"),
        pd.Timestamp("20130101 09:00:03"),
        pd.Timestamp("20130101 09:00:04"),
        pd.Timestamp("20130101 09:00:06"),
    ])

df["right"] = df.rolling("2s", closed="right").x.sum()  # default
df["both"] = df.rolling("2s", closed="both").x.sum()
df["left"] = df.rolling("2s", closed="left").x.sum()
df["neither"] = df.rolling("2s", closed="neither").x.sum()
df

## Groupby 
Creating [groupby](https://pandas.pydata.org/docs/user_guide/groupby.html#) object

In [ ]:
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    }
)
df.groupby("A")
df.groupby("A").groups
df.groupby("A").sum()

Getting a single group

In [ ]:
df.groupby("A").get_group('foo')
df.groupby(["A","B"]).get_group(('foo','one'))

Groupby with MultiIndex

In [ ]:
arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],]
index = pd.MultiIndex.from_arrays(arrays, names=["first", "second"])
df2 = pd.DataFrame({"A": [1, 1, 1, 1, 2, 2, 3, 3], "B": np.arange(8)}, index=index)
df2

df2.groupby(level=1).sum()
df2.groupby(level="second").sum()
df2.groupby("second").sum()
df2.groupby(["first","second"]).sum()

Groupby with index levels and columns

In [ ]:
df2.groupby([pd.Grouper(level=1), "A"]).sum()
df2.groupby(["second","A"]).sum()

Selecting columns

In [ ]:
df.groupby("A")["C"].sum()
df.groupby("A")["D","C"].sum()

Iterationg through groups

In [ ]:
for name, group in df2.groupby("A"):
    print(name)
    print(group)

## Groupby aggregate

Once the GroupBy object has been created, several methods are available to perform a computation on the grouped data. An obvious one is aggregation via the [aggregate()](https://pandas.pydata.org/docs/user_guide/groupby.html#aggregation) or equivalently agg() method.

In [ ]:
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    })
df.groupby(["A", "B"]).sum()
df.groupby(["A", "B"]).aggregate(np.sum)
df.groupby(["A", "B"]).agg(np.sum)
df.groupby(["A", "B"]).agg("sum")

Removing MultiIndex

In [ ]:
df.groupby(["A", "B"], as_index=False).agg(np.sum)
# or
df.groupby(["A", "B"]).agg(np.sum).reset_index()

Basic [aggregating functions](https://pandas.pydata.org/docs/user_guide/groupby.html#aggregation) on groupby objects

size() returns a Series whose index are the group names and whose values are the sizes of each group

In [ ]:
df.groupby(["A", "B"]).size()

In [ ]:
df.groupby(["A", "B"]).describe()

Number of unique values of each group

In [ ]:
df.groupby(["A"]).nunique()

Some common aggregations, currently only sum, mean, std, and sem, have optimized Cython implementations

In [ ]:
df.groupby(["A", "B"]).mean()

Applying user defined function

In [ ]:
df.groupby('A').agg(lambda x: 1)
df.groupby('A').agg(lambda x: x.sum())
df.groupby('A').agg(lambda x: x.max() - x.min())

In [ ]:
animals = pd.DataFrame(
    {   "kind": ["cat", "dog", "cat", "dog"],
        "height": [9.1, 6.0, 9.5, 34.0],
        "weight": [7.9, 7.5, 9.9, 198.0],
    })
animals.groupby("kind")[["height"]].agg(lambda x: set(x))
animals.groupby("kind")[["height"]].agg(lambda x: x.astype(int).sum())

Applying multiple functions

In [ ]:
df.groupby("A")["C"].agg([np.sum, np.mean, np.std])

Renaming columns

In [ ]:
(
    df.groupby("A")["C"]
    .agg([np.sum, np.mean, np.std])
    .rename(columns={"sum": "foo", "mean": "bar", "std": "baz"})
)

Multiple functions on different columns

In [ ]:
df.groupby("A")["C"].agg([lambda x: x.max() - x.min(),
                          lambda x: x.median() - x.mean()])
df.groupby("A").agg({"C" : [lambda x: x.max() - x.min(),
                            lambda x: x.median() - x.mean()]})
df.groupby("A").agg({"C" : [lambda x: x.max() - x.min()],
                     "D" : [np.mean,np.std]})

Using [named aggregation](https://pandas.pydata.org/docs/user_guide/groupby.html#named-aggregation) - you can name columns yourself!

In [ ]:
g1 = df.groupby("A").agg(span_C = pd.NamedAgg(column = "C", aggfunc = lambda x: x.max() - x.min()),
                         mean_D = pd.NamedAgg(column = "D", aggfunc = np.mean),
                         std_D = pd.NamedAgg(column = "D", aggfunc = np.std))

# same as
g2 = df.groupby("A").agg(span_C = ("C", lambda x: x.max() - x.min()),
                         mean_D = ("D", np.mean),
                         std_D = ("D", np.std))

# or for a single column
g3 = df.groupby("A")["C"].agg(span_C = lambda x: x.max() - x.min())

g1
g2
g3

## Groupby transform

The [transform](https://pandas.pydata.org/docs/user_guide/groupby.html#transformation) method returns an object that is indexed the same (same size) as the one being grouped

In [ ]:
index = pd.date_range("10/1/1999", periods=1100)
ts = pd.Series(np.random.normal(0.5, 2, 1100), index)
ts = ts.rolling(window=100, min_periods=100).mean().dropna()

transformed = ts.groupby(lambda x: x.year).transform(
    lambda x: (x - x.mean()) / x.std())
transformed

Transformation functions that have lower dimension outputs are broadcast to match the shape of the input array

In [ ]:
ts.groupby(lambda x: x.year).transform(lambda x: x.max() - x.min())

Same as:

In [ ]:
max = ts.groupby(lambda x: x.year).transform("max")
min = ts.groupby(lambda x: x.year).transform("min")
max - min

Replacing missing data with group mean

In [ ]:
data_df = pd.DataFrame({"A":np.random.normal(0, 1, 1000),
                       "B":np.random.normal(0, 1, 1000),
                       "C":np.random.normal(0, 1, 1000)})
data_df['C'][data_df['C']<0] = np.nan
data_df

countries = np.array(["US", "UK", "GR", "JP"])
key = countries[np.random.randint(0, 4, 1000)]
grouped = data_df.groupby(key)
grouped.count()

transformed = grouped.transform(lambda x: x.fillna(x.mean()))
transformed.head()
transformed.groupby(key).count()

[Window and resample operations](https://pandas.pydata.org/docs/user_guide/groupby.html#window-and-resample-operations)

Getting mean of 4 previous elements with rolling()

In [ ]:
df_re = pd.DataFrame({"A": [1] * 10 + [5] * 10, "B": np.arange(20)})
df_re
df_re.groupby("A").rolling(4).B.mean()

The expanding() method will accumulate a given operation (sum() in the example) for all the members of each particular group

In [ ]:
df_re.groupby("A").expanding().sum()

You can to use the resample() method to get a daily frequency in each group of your dataframe and complete the missing values with the ffill() method

In [ ]:
df_re = pd.DataFrame({"date": pd.date_range(start="2016-01-01", periods=4, freq="W"),
                      "group": [1, 1, 2, 2],
                      "val": [5, 6, 7, 8]}).set_index("date")
df_re
df_re.groupby("group").resample("1D").ffill()

## Groupby filter

The [filter](https://pandas.pydata.org/docs/user_guide/groupby.html#filtration) method returns a subset of the original object

Take only elements that belong to groups with a group sum greater than 2

In [ ]:
sf = pd.Series([1, 1, 2, 3, 3, 3])
sf.groupby(sf).filter(lambda x: x.sum() > 2)

Filtering out elements that belong to groups with only a couple members

In [ ]:
dff = pd.DataFrame({"A": np.arange(8), "B": list("aabbbbcc")})
dff.groupby("B").filter(lambda x: len(x) > 2)

Return a like-indexed objects where the groups that do not pass the filter are filled with NaNs

In [ ]:
dff.groupby("B").filter(lambda x: len(x) > 2, dropna=False)

For DataFrames with multiple columns, filters should explicitly specify a column as the filter criterion

In [ ]:
dff["C"] = np.arange(8)
dff.groupby("B").filter(lambda x: len(x["C"]) > 2)

The nlargest and nsmallest methods work on Series style groupbys

In [ ]:
s = pd.Series([9, 8, 7, 5, 19, 1, 4.2, 3.3])
g = pd.Series(list("abababab"))
gb = s.groupby(g)
gb.nlargest(1)
gb.nsmallest(1)

## Groupby apply

[Apply](https://pandas.pydata.org/docs/user_guide/groupby.html#flexible-apply) function can be substituted for both aggregate and transform in many standard use cases

In [ ]:
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    })

# output is a series
df.groupby("A")["C"].apply(lambda x: x.describe())
# or (output is a dataframe)
df.groupby("A").apply(lambda x: x["C"].describe())

Operations with multiple columns

In [ ]:
df.groupby("A").apply(lambda x: x["C"] + x["D"])

Changing dimension

In [ ]:
def f(group):
    return pd.DataFrame({'original': group,
                         'demeaned': group - group.mean()})

df.groupby('A')['C'].apply(f)

In [ ]:
def f(x):
    return pd.Series([x, x ** 2], index=["x", "x^2"])
s = pd.Series(np.random.rand(5))
s.apply(f)

## Other useful features

Taking the first or the last rows of each group

In [ ]:
df.groupby("A").head(1) # first
df.groupby("A").tail(1) # last
df.groupby("A").nth(0) # first
df.groupby("A").nth(-1) # last
df.groupby("A").first() # first
df.groupby("A").last() # last
df.groupby("A").nth(-1,dropna="any") # last excluding NaN
df.groupby("A").nth([0,-1]) # first and last

To see the order in which each row appears within its group, use the cumcount method

In [ ]:
dfg = pd.DataFrame(list("aaabba"), columns=["A"])
dfg.groupby("A").cumcount()

To see the ordering of the groups you can use ngroup(). The numbers given to the groups match the order in which the groups would be seen when iterating over the groupby object.

In [ ]:
dfg.groupby("A").ngroup()
dfg.groupby("A").ngroup(ascending=False)

Plotting

In [ ]:
np.random.seed(1234)
df = pd.DataFrame(np.random.randn(50, 2))
df["g"] = np.random.choice(["A", "B"], size=50)

# adding 3 to each element in column 1 if it belongs to group B in g column
df.loc[df["g"] == "B", 1] += 3
df.groupby("g").boxplot()

[Piping](https://pandas.pydata.org/docs/user_guide/groupby.html#piping-function-calls)

In [ ]:
n = 1000
df = pd.DataFrame(
    {
        "Store": np.random.choice(["Store_1", "Store_2"], n),
        "Product": np.random.choice(["Product_1", "Product_2"], n),
        "Revenue": (np.random.random(n) * 50 + 10).round(2),
        "Quantity": np.random.randint(1, 10, size=n)})

df.groupby(["Store", "Product"]).pipe(lambda x: x['Revenue'].sum() / x['Quantity'].sum()).unstack().round(2)
df.groupby(["Store", "Product"]).pipe(lambda x: x['Revenue'].sum() / x['Quantity'].sum())
df.groupby(["Store", "Product"]).pipe(lambda x: x.mean())
df.groupby(["Store", "Product"]).apply(lambda x: x['Revenue'].sum() / x['Quantity'].sum())


Regrouping columns of a DataFrame according to their sum, and summing the aggregated ones.

In [ ]:
df = pd.DataFrame({"a": [1, 0, 0], "b": [0, 1, 0], "c": [1, 0, 0], "d": [2, 3, 4]})
df.groupby(df.sum(),axis=1).sum()

Groupby by indexer to ‘resample’ data.

Grouping data into bins of 5 (df.index // 5)

In [ ]:
df = pd.DataFrame(np.random.randn(10, 2))
df.index // 5
df.groupby(df.index // 5).std()

In [ ]:
Group DataFrame columns, compute a set of metrics and return a named Series

In [ ]:
df = pd.DataFrame(
    {
        "a": [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
        "b": [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
        "c": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
        "d": [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],})
def compute_metrics(x):
    result = {"b_sum": x["b"].sum(), "c_mean": x["c"].mean()}
    return pd.Series(result, name="metrics")
result = df.groupby("a").apply(compute_metrics)
result
result.stack()

# Excercises

**Excel table S2**

Calculate correlation between coverage (Tend) and ploidy (Tend) per each cross and plot scatterplot of two variables in each cross (function corr() with groupby calculates pairwise correlations for all pairs of columns)

**Excel table S4**

For each Cross and Scaffold calculate mean mutation rate: N mutations/(Length (bp) x copy number * N generations)

Either 1) get rate per line -> group -> get mean,
or 2) group -> calculate rate on summed mutations and lengths. 
Some lines have length == 0

**Excel table S2**

Calculate Z-scores for 'Mean depth of coverage at Tini' ( (x - x.mean())/x.std() ) standardized across each cross and add to dataframe as a new column 'z-score' (use transform)

**Excel table S5**

Calculate Ts/Tv for each cross and plot as a barplot